In [1]:
import warnings
warnings.filterwarnings("ignore")
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Dashapp to display implied volatility vs time and price using a constant underlying price
* **This requires plotly 4.0 or greater** *

Calculate the implied volatility of each days options as if the underlying futures contract were always some fixed value. For example, if the actual futures is 100, make that price 50, and then change all of the strike prices so that the strike prices are relative to the value 50.  

1. The 100 call would be a 50 dollar call.  
2. The 95 put would be a 45 dollar put.  
3. The 105 call would be a 55 dollar call


In [2]:
import pandas as pd
import numpy as np

import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
from tqdm import tqdm,tqdm_notebook
from dashapp import dashapp2 as dashapp
import dash_html_components as html
import dash_core_components as dcc
import pyarrow as pa
import redis
import pdb

In [3]:
redis_port = 6379
redis_db = redis.Redis(host = 'localhost',port=6379,db=0)

In [4]:
def get_redis_df(key):
    context = pa.default_serialization_context()
    df = context.deserialize(redis_db.get(key))
    return df

### Show graphs

In [5]:
commod = 'CL'
year = 2020
cols = ['settle_date','iv','fprice']
df_cuv_implied = get_redis_df('df_cuv_implied')
symbols = df_cuv_implied[df_cuv_implied.symbol.str.slice(0,2)==commod].symbol.unique()
symbols = [s for s in symbols if int(s[-2:])+2000 == year]
for symbol in [k for k in symbols]:
    df_to_graph = df_cuv_implied[df_cuv_implied.symbol==symbol].iloc[:-1][cols]
    fig = dashapp.plotly_plot(
        df_in=df_to_graph,x_column='settle_date',yaxis2_cols=['fprice'],
        y_left_label='Implied Volatility',y_right_label='Futures Close',
        plot_title=f"{symbol} Pseudo-Volatility vs Price"
    )
    iplot(fig)


### Methods to create dashapp

In [6]:
def make_page_title(title_text,div_id=None,html_container=None,parent_class=None,
                   panel_background_color='#CAE2EB'):
    par_class = parent_class
    if par_class is None:
        par_class = dashapp.pnnm
    htmc = html_container
    if htmc is None:
        htmc = html.H2
        
    title_parts = title_text.split('\n')
    

    title_list = [htmc(tp,className=dashapp.pnncnm) for tp in title_parts]
    r = dashapp.multi_row_panel(title_list,
                 parent_class=par_class,
                 div_id=div_id,
                 panel_background_color=panel_background_color) 
    return r   


In [65]:
_create_dropdown_callback = lambda df,comp_id,col,init_value,multi: dashapp.make_dropdown(
            df,comp_id,col,current_value=init_value,multi=multi)

_inputbox_style = {"font-size":"18px","text-align":"center","position":"relative",
    "display":"inline-block","width":"130px","height":"45px"}
def _create_inputbox_callback(df,comp_id,col,init_value,multi=None): 
    inputbox = dcc.Input(
        id='comp_id',value=init_value,
        style=_inputbox_style,debounce=True,step=.001
    )    
    
    
# def components_from_df(df,columns,d_prefix,init_values=None,multi=False,
#                       create_component_callback=None):
#     ccc = _create_dropdown_callback
#     if create_component_callback is not None:
#         ccc = create_component_callback
def components_from_df(df,columns,d_prefix,init_values=None,multi=False):
    dict_component = {}
    for i in range(len(columns)):
        col = columns[i]
        init_value = None if init_values is None else init_values[i]
        if multi:
            init_value = [init_value]
        comp_id = f"{d_prefix}_{col}"
        comp = _create_dropdown_callback(
            df,comp_id,col,init_value,multi)
        dict_component[col] = comp
    return dict_component        
        


In [66]:
_query_operators_options = [
    {'label':'','value':''},
    {'label':'=','value':'=='},
    {'label':'!=','value':'!='},
    {'label':'>','value':'>'},
    {'label':'<','value':'<'},
    {'label':'>=','value':'>='},
    {'label':'<=','value':'<='},
    {'label':'free','value':''},
] 

class FilterDfComponent(html.Div):
    def __init__(self,comp_id,col,text=None):
        super(FilterDfComponent,self).__init__(id=comp_id)
        self.col = col
        div_text = col if text is None else text
        self.col_div = dashapp.make_text_centered_div(div_text)
        dd_id = f"ddop_{comp_id}"
        self.operator_dd = dcc.Dropdown(id=dd_id,options=_query_operators_options,value='')
        input_id = f"inputop_{comp_id}"
        self.operand_input = dcc.Input(id=input_id,value='',debounce=True)
        
        mcp =  dashapp.multi_column_panel(
            [self.col_div,self.operator_dd,self.operand_input],
            grid_template='1fr 1fr 1fr'
        )
        self.children = mcp#.children
        
class _FilterDescriptor():
    def __init__(self,column,init_value,text):
        self.column=column
        self.init_value=init_value
        self.text=text



In [67]:
def create_dashtable_page(
    df,
    dt_id,
    filter_dd_list,
    get_data_callback,
    multi=False,
    loading_full_screen=True,
    title=None):
    
#     dd_prefix = f"{dt_id}_dd_"
    filter_columns = [fdd.column for fdd in filter_dd_list]
    init_values = [fdd.init_value for fdd in filter_dd_list]
    text_values = [fdd.text for fdd in filter_dd_list]
    filter_components = [FilterDfComponent(f"{dt_id}_{[fdd.column]}_fc",fdd.column,fdd.text) for fdd in filter_dd_list]
    
    
    data_store = dcc.Store(id=f'{dt_id}_data_store')
    data_store_loading = dcc.Loading(
        id='data_store_loading',children=[data_store],fullscreen=loading_full_screen)
    
    # callback to populate data_store's dataframe
    id_df = f'df_{dt_id}'
    def _filter_data(input_data):
        df = get_data_callback()
        # Get len of input_data array.  
        # Half of that array are from dcc.Inputs that you 
        #   specify in the input_tuple_list argument to the DashLink constructor.
        # The other half of that array are from dcc.Dropdowns that you
        #   specify in the state_tuple_list argument to the DashLink constructor.
        input_data_len = len(input_data)
        num_inputs = int(input_data_len/2)
        for i in range(num_inputs):
            dropdown_value = input_data[i+num_inputs]
            if dropdown_value == '':
                continue
            input_value = input_data[i]
            if (input_value is not None) and (dropdown_value is not None):
                col = filter_columns[i]
                # see if col is numeric
                needs_quotes = False
                try:
                    colsum = df[col].astype(float).sum()
                except:
                    needs_quotes = True
                ipv = "''" if input_value is None else input_value
                ipv = f"'{ipv}'" if needs_quotes else ipv
                filter_query = f"{col} {dropdown_value} {ipv}"
                print(f"_filter_query: {filter_query}")
                df = df.query(filter_query)
        return [{id_df:df.to_dict('rows')}]
    
    filter_inputs = [(fc.operand_input,'value') for fc in filter_components]
    filter_dropdowns = [(fc.operator_dd,'value') for fc in filter_components]
    
    data_store_link = dashapp.DashLink(
        filter_inputs,[(data_store,'data')],
        io_callback=_filter_data,
        state_tuple_list=filter_dropdowns
    )
    
    # create dashtable, using the data_store
    iv_list = init_values if init_values is not None else [None for _ in filter_columns]
    df_init = get_data_callback()
    dt_values,dt_nav_link = dashapp.make_dashtable(
        dt_id,df_in=df_init,max_width=None,
        input_store = data_store,
        input_store_key=id_df,
    )
    
    
    title_row = None
    if title is not None:
        title_row = make_page_title(title,div_id=f"{dt_id}_title_row",html_container=html.H3)  
    
    # make the multi column panel for the filter row
    filter_htmls = [
#         dashapp.multi_column_panel(
        dashapp.multi_row_panel(
#             [html.Div(text_values[i]),filter_components[i]],
#             grid_template=['1fr 1fr'],parent_class=None,
            [filter_components[i]],
            grid_template=['1fr'],parent_class=None,            
        ) for i in range(len(filter_dd_list))
    ]
    
    fr_gt = [' '.join(['1fr' for _ in range(len(filter_htmls))])]
    filter_row = dashapp.multi_column_panel(
        filter_htmls,parent_class=dashapp.pn,div_id=f'{dt_id}_filter_row',
        grid_template=fr_gt
    )
    
    
    # row with dashtable
    dt_values_row = html.Div(dt_values,style={'width':'90vw'})
    # row 7 col 2
    all_rows = [filter_row,title_row,dt_values_row,data_store_loading]
    all_links = [dt_nav_link,data_store_link]
    return all_rows,all_links,
    


In [68]:
def get_cuv_imp_from_redis():
    df_cuv_imp2 = get_redis_df('df_cuv_implied')
    df_cuv_imp2['commod'] = df_cuv_imp2.symbol.str.slice(0,2)
    df_cuv_imp2['contract_year'] = df_cuv_imp2.symbol.str.slice(-2,).astype(int) + 2000
    return df_cuv_imp2
        

In [94]:
# import importlib
# importlib.reload(dashapp)

<module 'dashapp.dashapp2' from '/Users/bperlman1/Documents/billybyte/pyliverisk/dashapp/dashapp/dashapp2.py'>

In [70]:
if __name__=='__main__':
    panel_color = '#FFFFFA'
    all_links = []    
    
    # Create an instance of DashApp, which holds all of the html and dcc elements, 
    #    as well as all of the DashLinks, and finally creates the instance of Dash.app
    dap = dashapp.DashApp()
    
    # *********** Assemble all of he rows and columns below ***************

    # ************ Create row 1 (the main title) *******************
    app_title = """Show ATM Implied Volatilities
using a
Constant Underlying Price"""
    r1 = make_page_title(app_title,div_id='r1',html_container=html.H3)                  

    # *************** Create other rows ****************************
    df_init = get_cuv_imp_from_redis()
    filter_ddd_list = [
        _FilterDescriptor('commod','CL','Commodity: '),
        _FilterDescriptor('contract_year',2020,'Contract Year: ')
    ]
    rows_2_4,all_links = create_dashtable_page(
        df_init,'df_cuv_implied',filter_ddd_list,get_cuv_imp_from_redis,
        title = "ATM Vol vs Price Data"
    )
    
       
    all_rows = html.Div([r1]+rows_2_4)

    # Add all of the DashLinks to the DashApp instance (dap)
#     pdb.set_trace()
    dap.add_links(all_links)
    # Create the dash app object by calling the create_app method of dap (the instance of DashApp)
    dap.create_app(all_rows,app_title='Constant Underlying Vol',app_port=8804)
    
    

2020-06-24 16:09:40,346 - root - DEBUG - df_cuv_implied entering create_dt_div
2020-06-24 16:09:40,350 - root - DEBUG - df_cuv_implied exiting create_dt_div
2020-06-24 16:09:40,365 - root - INFO - This app will run at the URL: http://127.0.0.1:8804


 * Serving Flask app "dashapp.dashapp2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-06-24 16:09:40,374 - werkzeug - INFO -  * Running on http://127.0.0.1:8804/ (Press CTRL+C to quit)
2020-06-24 16:09:43,833 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:09:43] "GET / HTTP/1.1" 200 -
2020-06-24 16:09:43,968 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:09:43] "GET /_dash-layout HTTP/1.1" 200 -
2020-06-24 16:09:43,970 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:09:43] "GET /_dash-dependencies HTTP/1.1" 200 -
2020-06-24 16:09:44,760 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:09:44] "POST /_dash-update-component HTTP/1.1" 200 -
2020-06-24 16:09:45,198 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:09:45] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:0 page_size: 20
_filter_query: commod == 'CB'


2020-06-24 16:10:03,446 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:10:03] "POST /_dash-update-component HTTP/1.1" 200 -
2020-06-24 16:10:03,626 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:10:03] "POST /_dash-update-component HTTP/1.1" 200 -


_dash_table_update_paging_closure: page_current:0 page_size: 20


2020-06-24 16:10:08,978 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:10:08] "POST /_dash-update-component HTTP/1.1" 200 -
2020-06-24 16:10:09,026 - werkzeug - INFO - 127.0.0.1 - - [24/Jun/2020 16:10:09] "POST /_dash-update-component HTTP/1.1" 200 -


_filter_query: commod == 'CB'
_filter_query: contract_year == 2019
_dash_table_update_paging_closure: page_current:0 page_size: 20


In [95]:
df_test = get_cuv_imp_from_redis()
df_test = df_test[df_test.symbol.str.slice(0,2)=='CL']


c= 'settle_date'
g = dashapp.generate_sub_dfs(df_test,c,10)
for df_sub in g:
    y_defs = [['name','x_column','row','col','is_secondary','yaxis_title']]
    symbols = df_sub.symbol.unique()
    df_sub_to_plot = pd.DataFrame({'settle_date':df_sub.settle_date.unique()})
    for symbol in symbols:
        y_defs.append([symbol,'settle_date',1,1,False,'Implied Vol'])
        df_sub_to_plot_this_symbol = df_sub[df_sub.symbol==symbol][['settle_date','iv']]
        df_sub_to_plot_this_symbol = df_sub_to_plot_this_symbol.rename(columns={'iv':symbol})
        df_sub_to_plot = df_sub_to_plot.merge(
            df_sub_to_plot_this_symbol,on='settle_date',how='left')
        
        
    df_figure = pd.DataFrame(y_defs[1:],columns=y_defs[0])  
    yyyymmdd_beg = df_sub_to_plot.settle_date.min()
    yyyymmdd_end = df_sub_to_plot.settle_date.max()
    title = f'Implied Vol vs Date for all Contracts - {yyyymmdd_beg} = {yyyymmdd_end}'
    iplot(
        dashapp.plotly_subplots(
            df_sub_to_plot,
            df_figure,title=title
        )
    )

In [63]:
display.display(df_test.columns.values)
display.display(df_test.symbol.unique())


array(['settle_date', 'iv', 'fprice', 'symbol', 'commod', 'year'],
      dtype=object)

array(['CBG11', 'CBH11', 'CBJ11', 'CBK11', 'CBM11', 'CBN11', 'CBQ11',
       'CBU11', 'CBV11', 'CBX11', 'CBZ11', 'CBF12', 'CBG12', 'CBH12',
       'CBJ12', 'CBK12', 'CBM12', 'CBN12', 'CBQ12', 'CBU12', 'CBV12',
       'CBX12', 'CBZ12', 'CBF13', 'CBG13', 'CBH13', 'CBJ13', 'CBK13',
       'CBM13', 'CBN13', 'CBQ13', 'CBU13', 'CBV13', 'CBX13', 'CBZ13',
       'CBF14', 'CBG14', 'CBH14', 'CBJ14', 'CBK14', 'CBM14', 'CBN14',
       'CBQ14', 'CBU14', 'CBV14', 'CBX14', 'CBZ14', 'CBF15', 'CBG15',
       'CBH15', 'CBJ15', 'CBK15', 'CBM15', 'CBN15', 'CBQ15', 'CBU15',
       'CBV15', 'CBX15', 'CBZ15', 'CBF16', 'CBG16', 'CBH16', 'CBJ16',
       'CBK16', 'CBM16', 'CBN16', 'CBQ16', 'CBU16', 'CBV16', 'CBX16',
       'CBZ16', 'CBF17', 'CBG17', 'CBH17', 'CBJ17', 'CBK17', 'CBM17',
       'CBN17', 'CBQ17', 'CBU17', 'CBV17', 'CBX17', 'CBZ17', 'CBF18',
       'CBG18', 'CBH18', 'CBJ18', 'CBK18', 'CBM18', 'CBN18', 'CBQ18',
       'CBU18', 'CBV18', 'CBX18', 'CBZ18', 'CBF19', 'CBG19', 'CBH19',
       'CBJ19', 'CBK

## END